In [1]:
from microphone import record_audio
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
import librosa
import matplotlib.mlab as mlab
from scipy.ndimage.morphology import generate_binary_structure
from scipy.ndimage.morphology import iterate_structure
from typing import Tuple, List
from numba import njit

In [3]:
import pickle

In [4]:
def micRecord(time=10):
    frames, rate = record_audio(time)
    return np.hstack([np.frombuffer(i, np.int16) for i in frames]), rate

In [5]:
def getFile(path):
    recorded_audio, sampling_rate = librosa.load(path, 
                                                 sr=44100, 
                                                 mono=True, 
                                                 offset=1.5, 
                                                 duration=20)
    return recorded_audio, sampling_rate

In [6]:
def userinput():
    while True:
        audioType = input("u for Upload, r for Record: ")
        if audioType == 'u':
            path = input("Enter path to file: ")
            samples, rate = getFile(path)
            break
        elif audioType == 'r':
            samples, rate = micRecord()
            break
        print("Invalid input. Try again.")
    # print(rate)
    
    return samples, rate

In [7]:
def ecdf(data):
    """Returns (x) the sorted data and (y) the empirical cumulative-proportion
    of each datum.
    
    Parameters
    ----------
    data : numpy.ndarray, size-N
    
    Returns
    -------
    Tuple[numpy.ndarray shape-(N,), numpy.ndarray shape-(N,)]
        Sorted data, empirical CDF values"""
    data = np.asarray(data).ravel()  # flattens the data
    y = np.linspace(1 / len(data), 1, len(data))  # stores the cumulative proportion associated with each sorted datum
    x = np.sort(data)
    return x, y

In [8]:
@njit
def _peaks(
    data_2d: np.ndarray, nbrhd_row_offsets: np.ndarray, nbrhd_col_offsets: np.ndarray, amp_min: float
) -> List[Tuple[int, int]]:
    """
    A Numba-optimized 2-D peak-finding algorithm.
    
    Parameters
    ----------
    data_2d : numpy.ndarray, shape-(H, W)
        The 2D array of data in which local peaks will be detected.
    nbrhd_row_offsets : numpy.ndarray, shape-(N,)
        The row-index offsets used to traverse the local neighborhood.
        
        E.g., given the row/col-offsets (dr, dc), the element at 
        index (r+dr, c+dc) will reside in the neighborhood centered at (r, c).
    
    nbrhd_col_offsets : numpy.ndarray, shape-(N,)
        The col-index offsets used to traverse the local neighborhood. See
        `nbrhd_row_offsets` for more details.
        
    amp_min : float
        All amplitudes equal to or below this value are excluded from being
        local peaks.
    
    Returns
    -------
    List[Tuple[int, int]]
        (row, col) index pair for each local peak location, returned in 
        column-major order
    """
    peaks = []  # stores the (row, col) locations of all the local peaks

    # Iterating over each element in the the 2-D data 
    # in column-major ordering
    #
    # We want to see if there is a local peak located at
    # row=r, col=c
    for c, r in np.ndindex(*data_2d.shape[::-1]):
        if data_2d[r, c] <= amp_min:
            # The amplitude falls beneath the minimum threshold
            # thus this can't be a peak.
            continue
        
        # Iterating over the neighborhood centered on (r, c) to see
        # if (r, c) is associated with the largest value in that
        # neighborhood.
        #
        # dr: offset from r to visit neighbor
        # dc: offset from c to visit neighbor
        for dr, dc in zip(nbrhd_row_offsets, nbrhd_col_offsets):
            if dr == 0 and dc == 0:
                # This would compare (r, c) with itself.. skip!
                continue

            if not (0 <= r + dr < data_2d.shape[0]):
                # neighbor falls outside of boundary.. skip!
                continue

            if not (0 <= c + dc < data_2d.shape[1]):
                # neighbor falls outside of boundary.. skip!
                continue

            if data_2d[r, c] < data_2d[r + dr, c + dc]:
                # One of the amplitudes within the neighborhood
                # is larger, thus data_2d[r, c] cannot be a peak
                break
        else:
            # if we did not break from the for-loop then (r, c) is a local peak
            peaks.append((r, c))
    return peaks

In [9]:
def local_peak_locations(data_2d: np.ndarray, neighborhood: np.ndarray, amp_min: float):
    """
    Defines a local neighborhood and finds the local peaks
    in the spectrogram, which must be larger than the specified `amp_min`.
    
    Parameters
    ----------
    data_2d : numpy.ndarray, shape-(H, W)
        The 2D array of data in which local peaks will be detected
    
    neighborhood : numpy.ndarray, shape-(h, w)
        A boolean mask indicating the "neighborhood" in which each
        datum will be assessed to determine whether or not it is
        a local peak. h and w must be odd-valued numbers
        
    amp_min : float
        All amplitudes at and below this value are excluded from being local 
        peaks.
    
    Returns
    -------
    List[Tuple[int, int]]
        (row, col) index pair for each local peak location, returned
        in column-major ordering.
    
    Notes
    -----
    The local peaks are returned in column-major order, meaning that we 
    iterate over all nbrhd_row_offsets in a given column of `data_2d` in search for
    local peaks, and then move to the next column.
    """

    # We always want our neighborhood to have an odd number
    # of nbrhd_row_offsets and nbrhd_col_offsets so that it has a distinct center element
    assert neighborhood.shape[0] % 2 == 1
    assert neighborhood.shape[1] % 2 == 1
    
    # Find the indices of the 2D neighborhood where the 
    # values were `True`
    #
    # E.g. (row[i], col[i]) stores the row-col index for
    # the ith True value in the neighborhood (going in row-major order)
    nbrhd_row_indices, nbrhd_col_indices = np.where(neighborhood)
    

    # Shift the neighbor indices so that the center element resides 
    # at coordinate (0, 0) and that the center's neighbors are represented
    # by "offsets" from this center element.
    #
    # E.g., the neighbor above the center will has the offset (-1, 0), and 
    # the neighbor to the right of the center will have the offset (0, 1).
    nbrhd_row_offsets = nbrhd_row_indices - neighborhood.shape[0] // 2
    nbrhd_col_offsets = nbrhd_col_indices - neighborhood.shape[1] // 2

    return _peaks(data_2d, nbrhd_row_offsets, nbrhd_col_offsets, amp_min=amp_min)

In [10]:
def find_min_amp(spectrogram, amp_threshold):
    log_S = np.log(spectrogram).ravel()  # flattened array
    ind = round(len(log_S) * amp_threshold)
    cutoff_log_amplitude = np.partition(log_S, ind)[ind]
    return cutoff_log_amplitude

In [44]:
def peak_extract(samples, sampling_rate, *, amp_threshold=0.75, neighborhood_rank=2, neighborhood_connectivity=1, neighborhood_iterations=20):
	
    """
    Extracts peaks from a spectrogram created from the sample data.
    
    Parameters
    ----------
    samples : numpy.ndarray
        Array of audio samples
	
	sampling_rate : int
		The sampling rate of the audio samples
        
    amp_threshold : float
        All amplitudes at and below this value are excluded from being local 
        peaks.
	neighborhood_rank : int
	neighborhood_connectivity : int
	neighborhood_iterations : int
    
    Returns
    -------
    List[Tuple[int, int]]
        (row, col) index pair for each local peak location, returned
        in column-major ordering.
    """
    
    time = np.arange(len(samples)) / sampling_rate

    base_structure = generate_binary_structure(neighborhood_rank,neighborhood_connectivity)
    neighborhood = iterate_structure(base_structure, neighborhood_iterations)

    spectrogram, freqs, times = mlab.specgram(
		samples,
		NFFT=4096,
		Fs=sampling_rate,
		window=mlab.window_hanning,
		noverlap=int(4096 / 2)
	)

    #spectrogram = np.clip(spectrogram, 10**-20, None)
    
#     amp_min = find_min_amp(spectrogram, amp_threshold)

#     return local_peak_locations(spectrogram, neighborhood, amp_min)
    return spectrogram

In [63]:
%matplotlib notebook

In [66]:
def getspec(samples, rate):
    spectrogram, freqs, times = mlab.specgram(
		samples,
		NFFT=4096,
		Fs=rate,
		window=mlab.window_hanning,
		noverlap=int(4096 / 2)
	)
    return spectrogram, freqs, times

In [24]:
def pressure(times: np.ndarray, *, amp: float, freq: float) -> np.ndarray:
    return amp * np.sin(2 * np.pi * freq * times)

In [116]:
import matplotlib.mlab as mlab

amplitude = 0.06  # Pascals
duration = 4 # seconds
sampling_rate = 44100 # Hz
n_samples = int(duration * sampling_rate) + 1

times = np.arange(n_samples) / sampling_rate  # seconds
freqc = 261.63  # Hz
freqb = 246.94
c_4 = pressure(times, amp=amplitude, freq=freqc)  # Pascals
b_3 = pressure(times, amp=amplitude, freq=freqb)

music = np.concatenate((c_4,b_3))

spec, freq, time = getspec(music, sampling_rate)
freq = list(freq)

maxfreqs = freqs[np.argmax(spec, axis=0)]

timefreq = [(i,[j]) for i,j in enumerate(maxfreqs)]
timefreq = dict(timefreq)
print(timefreq)

with open("timefreq.pkl", 'wb') as f:
    pickle.dump(timefreq,f)

{0: [258.3984375], 1: [258.3984375], 2: [258.3984375], 3: [258.3984375], 4: [258.3984375], 5: [258.3984375], 6: [258.3984375], 7: [258.3984375], 8: [258.3984375], 9: [258.3984375], 10: [258.3984375], 11: [258.3984375], 12: [258.3984375], 13: [258.3984375], 14: [258.3984375], 15: [258.3984375], 16: [258.3984375], 17: [258.3984375], 18: [258.3984375], 19: [258.3984375], 20: [258.3984375], 21: [258.3984375], 22: [258.3984375], 23: [258.3984375], 24: [258.3984375], 25: [258.3984375], 26: [258.3984375], 27: [258.3984375], 28: [258.3984375], 29: [258.3984375], 30: [258.3984375], 31: [258.3984375], 32: [258.3984375], 33: [258.3984375], 34: [258.3984375], 35: [258.3984375], 36: [258.3984375], 37: [258.3984375], 38: [258.3984375], 39: [258.3984375], 40: [258.3984375], 41: [258.3984375], 42: [258.3984375], 43: [258.3984375], 44: [258.3984375], 45: [258.3984375], 46: [258.3984375], 47: [258.3984375], 48: [258.3984375], 49: [258.3984375], 50: [258.3984375], 51: [258.3984375], 52: [258.3984375], 53

In [56]:
for i in range(sorted(peak_dict.keys())[-1]):
    if i not in peak_dict.keys():
        peak_dict[i] = []
        
        

In [57]:
peak_dict = dict(sorted(peak_dict.items()))

In [58]:
for i in peak_dict.values():
    for f in i:
        if f == 0:
            i.remove(0)
            

In [50]:
peak_dict

{0: [675, 802, 1580, 1706],
 1: [397, 843, 872, 1149],
 2: [1303, 1348],
 3: [1107, 2013],
 4: [648, 1368, 1688],
 5: [1010],
 6: [285, 414, 467, 1266, 1418, 1669, 1806],
 7: [564, 940, 1769],
 8: [503],
 9: [202, 314, 606, 1509],
 10: [1247, 1971],
 11: [],
 12: [],
 13: [3],
 14: [773, 1205],
 15: [],
 16: [341, 1163],
 17: [],
 18: [1357, 1458, 1638, 1731, 1902],
 19: [885, 1024, 1275],
 20: [1557, 1820],
 21: [],
 22: [],
 23: [1434, 1678],
 24: [275, 732, 787],
 25: [999, 1574],
 26: [369, 536],
 27: [438, 760],
 28: [],
 29: [814],
 30: [606],
 31: [564, 1079, 1374],
 32: [202],
 33: [843, 982, 1126, 1395, 1602],
 34: [94, 1491],
 35: [],
 36: [],
 37: [348, 398, 773, 2041],
 38: [1693],
 39: [1637],
 40: [966, 1163, 1513, 1819],
 41: [],
 42: [],
 43: [],
 44: [261, 940, 1653],
 45: [],
 46: [467, 1709, 1754, 2013],
 47: [648, 1482],
 48: [1247],
 49: [784, 1010],
 50: [],
 51: [313, 1107, 1284, 1617],
 52: [1546, 1775],
 53: [355, 1795, 1834],
 54: [],
 55: [815],
 56: [49],
 5

In [59]:
savePeaks(peak_dict)